In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [3]:
toner_url = []
for i in range (1, 29):
    url = f'https://www.yesstyle.com/en/women-facial-toners/list.html/bcc.14337_bpt.46#/bt=37&sb=136&bpt=46&pn={i}&s=10&bcc=14337&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    toner_list = soup.find_all('div', class_='itemContainer')
    for item in toner_list:
        for link in item.find_all('a', href=True): 
            toner_url.append(link['href'])


In [4]:
print(len(toner_url))

1008


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [51]:
"""
test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text
"""

In [6]:
scrubs_url[0]

'https://www.yesstyle.com/en/the-face-shop-real-nature-face-mask-1pc-20-types-20g-calendula/info.html/pid.1061020764'

In [5]:
toners = []

In [12]:
for link in toner_url[860:1008]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text.strip() #ingredients
        rating = soup.find('div', class_='satisfied').text.strip()
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    toner_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    toners.append(toner_info)

In [13]:
print(len(toners))

1008


In [12]:
toners[496]

IndexError: list index out of range

## 3. PASARLO A DATAFRAME

In [14]:
toners_df = pd.DataFrame(toners)
toners_df

,name,rating,ingredients
0,"\nSOME BY MI - AHA, BHA, PHA 30 Days Miracle T...","95.5% customers satisfied|4,200 reviews","Water, Butylene Glycol, Dipropylene Glycol, Gl..."
1,\nI'm from - Rice Toner\n,"97.3% customers satisfied|2,151 reviews","Oryza Sativa (Rice) Extract (77.78%), Methylpr..."
2,\nPyunkang Yul - Essence Toner 200ml\n,"97.2% customers satisfied|3,135 reviews","Astragalus Membranaceus Root Extract, 1,2-Hexa..."
3,\nCOSRX - AHA/BHA Clarifying Treatment Toner\n,"95.6% customers satisfied|5,259 reviews","Water, Salix Alba (Willow) Bark Water, Pyrus M..."
4,"\nDear, Klairs - Supple Preparation Facial Ton...","97.9% customers satisfied|3,709 reviews","Aqua (Water), Butylene Glycol, Dimethyl Sulfon..."
...,...,...,...
1003,\nManyo - Bifida Ampoule Mist\n,100% customers satisfied|6 reviews,"Paeonia Lactiflora Bark/Sap Extract, Bifida Fe..."
1004,\nMISSHA - Time Revolution Artemisia Treatment...,96.1% customers satisfied|137 reviews,Artemisia Annua Extract\n\nMore
1005,\nBenton - Snail Bee High Content Lotion 120ml\n,96.4% customers satisfied|241 reviews,"Aqua (Water), Butylene Glycol, Snail Secretion..."
1006,\nSK-II - Facial Treatment Clear Lotion 30ml\n,87.3% customers satisfied|120 reviews,"Water, galactomyces culture solution (Gyunyu),..."


In [15]:
toner = toners_df.to_csv('yesstyle_data/toners.csv')